Mugs
Data Engineer internship

ASSIGNMENT

NAME- Khushi Tolani

EMAIL ID- kkhushitolanii@gmail.com

PHONE NO.- 8318094789

In [1]:
!wget https://physionet.org/static/published-projects/mimic-iv-fhir-demo/mimic-iv-clinical-database-demo-on-fhir-2.0.zip

--2024-05-07 22:20:54--  https://physionet.org/static/published-projects/mimic-iv-fhir-demo/mimic-iv-clinical-database-demo-on-fhir-2.0.zip
Resolving physionet.org (physionet.org)... 18.13.52.205
Connecting to physionet.org (physionet.org)|18.13.52.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121863908 (116M) [application/zip]
Saving to: ‘mimic-iv-clinical-database-demo-on-fhir-2.0.zip’

mimic-iv-clinical-d 100%[===================>] 116.22M   269KB/s    in 7m 25s  

2024-05-07 22:28:20 (267 KB/s) - ‘mimic-iv-clinical-database-demo-on-fhir-2.0.zip’ saved [121863908/121863908]



In [2]:
!unzip mimic-iv-clinical-database-demo-on-fhir-2.0.zip -d mimic_fhir_dataset

Archive:  mimic-iv-clinical-database-demo-on-fhir-2.0.zip
  inflating: mimic_fhir_dataset/mimic-iv-clinical-database-demo-on-fhir-2.0/LICENSE.txt  
  inflating: mimic_fhir_dataset/mimic-iv-clinical-database-demo-on-fhir-2.0/SHA256SUMS.txt  
  inflating: mimic_fhir_dataset/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/Condition.ndjson  
  inflating: mimic_fhir_dataset/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/Encounter.ndjson  
  inflating: mimic_fhir_dataset/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/EncounterICU.ndjson  
  inflating: mimic_fhir_dataset/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/Location.ndjson  
  inflating: mimic_fhir_dataset/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/Medication.ndjson  
  inflating: mimic_fhir_dataset/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/MedicationAdministration.ndjson  
  inflating: mimic_fhir_dataset/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/MedicationAdminist

In [3]:
import json
import pandas as pd
import os

In [56]:
def read_ndjson(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

In [57]:
dataset_dir = '/content/mimic_fhir_dataset/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/'

patients = read_ndjson(dataset_dir + 'Patient.ndjson')

conditions = read_ndjson(dataset_dir + 'Condition.ndjson')

encounters = read_ndjson(dataset_dir + 'Encounter.ndjson')

encounters_icu = read_ndjson(dataset_dir + 'EncounterICU.ndjson')

In [58]:
from datetime import datetime
import csv

In [60]:
def get_condition_timestamp(condition, encounters_data):
    encounter_id = condition.get('encounter', {}).get('reference', '').split('/')[-1]
    for encounter in encounters_data:
        if encounter['id'] == encounter_id:
            start_time = encounter.get('period', {}).get('start')
            if start_time:
                timestamp = datetime.fromisoformat(start_time.replace('Z', '+00:00')).timestamp()
                return int(timestamp)
    return None

In [61]:
patient_conditions = {}
for condition in conditions:
    patient_id = condition.get('subject', {}).get('reference', '').split('/')[-1]
    if patient_id not in patient_conditions:
        patient_conditions[patient_id] = []
    condition_timestamp = get_condition_timestamp(condition, encounters)
    if condition_timestamp is not None:
        condition_data = {
            'pid': patient_id,
            'time': condition_timestamp,
            'code': condition.get('code', {}).get('coding', [{}])[0].get('code', ''),
            'description': condition.get('code', {}).get('coding', [{}])[0].get('display', '')
        }
        patient_conditions[patient_id].append(condition_data)

In [62]:
print("Step 2: Patient Conditions:")
for patient_id, conditions in patient_conditions.items():
    print(f"Patient ID: {patient_id}")
    for condition in conditions:
        print(condition)

Step 2: Patient Conditions:
Patient ID: b410dd44-7d65-56f9-974f-2751e8aa80e2
{'pid': 'b410dd44-7d65-56f9-974f-2751e8aa80e2', 'time': 5616018000, 'code': 'Z8546', 'description': 'Personal history of malignant neoplasm of prostate'}
{'pid': 'b410dd44-7d65-56f9-974f-2751e8aa80e2', 'time': 5387190060, 'code': 'V1072', 'description': "Personal history of hodgkin's disease"}
{'pid': 'b410dd44-7d65-56f9-974f-2751e8aa80e2', 'time': 5639393940, 'code': 'Z7902', 'description': 'Long term (current) use of antithrombotics/antiplatelets'}
{'pid': 'b410dd44-7d65-56f9-974f-2751e8aa80e2', 'time': 5426582400, 'code': '49390', 'description': 'Asthma, unspecified type, unspecified'}
{'pid': 'b410dd44-7d65-56f9-974f-2751e8aa80e2', 'time': 5426582400, 'code': '2724', 'description': 'Other and unspecified hyperlipidemia'}
{'pid': 'b410dd44-7d65-56f9-974f-2751e8aa80e2', 'time': 5465203800, 'code': 'V5866', 'description': 'Long-term (current) use of aspirin'}
{'pid': 'b410dd44-7d65-56f9-974f-2751e8aa80e2', 't

In [63]:
csv_filename = 'patient_conditions.csv'
csv_columns = ['pid', 'time', 'code', 'description']
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for patient_id, conditions in patient_conditions.items():
        for condition in conditions:
            writer.writerow(condition)

In [64]:
print("CSV File Generated:", csv_filename)

CSV File Generated: patient_conditions.csv


In [65]:
import pandas as pd

df = pd.read_csv(csv_filename)
df


,pid,time,code,description
0,b410dd44-7d65-56f9-974f-2751e8aa80e2,5616018000,Z8546,Personal history of malignant neoplasm of pros...
1,b410dd44-7d65-56f9-974f-2751e8aa80e2,5387190060,V1072,Personal history of hodgkin's disease
2,b410dd44-7d65-56f9-974f-2751e8aa80e2,5639393940,Z7902,Long term (current) use of antithrombotics/ant...
3,b410dd44-7d65-56f9-974f-2751e8aa80e2,5426582400,49390,"Asthma, unspecified type, unspecified"
4,b410dd44-7d65-56f9-974f-2751e8aa80e2,5426582400,2724,Other and unspecified hyperlipidemia
...,...,...,...,...
4176,94abdf17-f13a-5eae-aac0-eca407bbfadd,6243609900,Z23,Encounter for immunization
4177,94abdf17-f13a-5eae-aac0-eca407bbfadd,6243609900,K219,Gastro-esophageal reflux disease without esoph...
4178,5ddeb201-5de6-5177-a116-fa82ce8ad2f2,5545862220,Z85850,Personal history of malignant neoplasm of thyroid
4179,5ddeb201-5de6-5177-a116-fa82ce8ad2f2,5545862220,G40909,"Epilepsy, unspecified, not intractable, withou..."


In [66]:
df.head()

,pid,time,code,description
0,b410dd44-7d65-56f9-974f-2751e8aa80e2,5616018000,Z8546,Personal history of malignant neoplasm of pros...
1,b410dd44-7d65-56f9-974f-2751e8aa80e2,5387190060,V1072,Personal history of hodgkin's disease
2,b410dd44-7d65-56f9-974f-2751e8aa80e2,5639393940,Z7902,Long term (current) use of antithrombotics/ant...
3,b410dd44-7d65-56f9-974f-2751e8aa80e2,5426582400,49390,"Asthma, unspecified type, unspecified"
4,b410dd44-7d65-56f9-974f-2751e8aa80e2,5426582400,2724,Other and unspecified hyperlipidemia


In [67]:
df.tail()

,pid,time,code,description
4176,94abdf17-f13a-5eae-aac0-eca407bbfadd,6243609900,Z23,Encounter for immunization
4177,94abdf17-f13a-5eae-aac0-eca407bbfadd,6243609900,K219,Gastro-esophageal reflux disease without esoph...
4178,5ddeb201-5de6-5177-a116-fa82ce8ad2f2,5545862220,Z85850,Personal history of malignant neoplasm of thyroid
4179,5ddeb201-5de6-5177-a116-fa82ce8ad2f2,5545862220,G40909,"Epilepsy, unspecified, not intractable, withou..."
4180,5ddeb201-5de6-5177-a116-fa82ce8ad2f2,5545862220,D1802,Hemangioma of intracranial structures


In [68]:
df.shape

(4181, 4)

In [69]:
df.info

<bound method DataFrame.info of                                        pid        time    code  \
0     b410dd44-7d65-56f9-974f-2751e8aa80e2  5616018000   Z8546   
1     b410dd44-7d65-56f9-974f-2751e8aa80e2  5387190060   V1072   
2     b410dd44-7d65-56f9-974f-2751e8aa80e2  5639393940   Z7902   
3     b410dd44-7d65-56f9-974f-2751e8aa80e2  5426582400   49390   
4     b410dd44-7d65-56f9-974f-2751e8aa80e2  5426582400    2724   
...                                    ...         ...     ...   
4176  94abdf17-f13a-5eae-aac0-eca407bbfadd  6243609900     Z23   
4177  94abdf17-f13a-5eae-aac0-eca407bbfadd  6243609900    K219   
4178  5ddeb201-5de6-5177-a116-fa82ce8ad2f2  5545862220  Z85850   
4179  5ddeb201-5de6-5177-a116-fa82ce8ad2f2  5545862220  G40909   
4180  5ddeb201-5de6-5177-a116-fa82ce8ad2f2  5545862220   D1802   

                                            description  
0     Personal history of malignant neoplasm of pros...  
1                 Personal history of hodgkin's disease  
2     Long term (current) use of antithrombotics/ant...  
3                 Asthma, unspecified type, unspecified  
4                  Other and unspecified hyperlipidemia  
...                                                 ...  
4176                         Encounter for immunization  
4177  Gastro-esophageal reflux disease without esoph...  
4178  Personal history of malignant neoplasm of thyroid  
4179  Epilepsy, unspecified, not intractable, withou...  
4180              Hemangioma of intracranial structures  

[4181 rows x 4 columns]>

In [70]:
from google.colab import files

files.download(csv_filename)
print("Successfully downloaded the csv file :", csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully downloaded the csv file : patient_conditions.csv
